<a href="https://colab.research.google.com/github/Rajib2805/ICVL_logistics/blob/main/ICVL_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**THE OPTIMIZATION MODEL IS FRAMED WITH THE OBJECTIVE OF HAVING A MONTHLY VIEW AND DERIVE ESTIMATES FOR THE MONTH AHEAD**

**Even though optimization problems are all about contraints in the decision making process, it must be remembered that constraints must be strived to be changed for the better with better operation and managerial decisions. It must be assumed that no constrains are God created and everything is changeable.**

In [33]:
!pip install pulp
import pulp as p

In [34]:
#Defining the problem
Lp_prob = p.LpProblem('Problem', p.LpMaximize)  

# Create problem Variables  
CCs_135 = p.LpVariable('CCs_135', lowBound = 0) 
CCs_105 = p.LpVariable('CCs_105', lowBound = 0) 
TCs = p.LpVariable('TCs', lowBound = 0) 
ROM_mined_perday = p.LpVariable('ROM_mined_perday', lowBound = 0) 
ROM_feed_perday = p.LpVariable('ROM_feed_perday', lowBound = 0) 
ROM_135_feed_perday = p.LpVariable('ROM_135_feed_perday', lowBound = 0) 
ROM_105_feed_perday = p.LpVariable('ROM_105_feed_perday', lowBound = 0) 
CCp_135 = p.LpVariable('CCp_135', lowBound = 0) 
CCp_105 = p.LpVariable('CCp_105', lowBound = 0) 
TCp = p.LpVariable('TCp', lowBound = 0) 
CCro_135 = p.LpVariable('CCro_135', lowBound = 0) 
CCro_105 = p.LpVariable('CCro_105', lowBound = 0) 
TCro = p.LpVariable('TCro', lowBound = 0) 
CCra_135 = p.LpVariable('CCra_135', lowBound = 0) 
CCra_105 = p.LpVariable('CCra_105', lowBound = 0) 
TCra = p.LpVariable('TCra', lowBound = 0) 
CClera_135 = p.LpVariable('CClera_135', lowBound = 0) 
CClera_105 = p.LpVariable('CClera_105', lowBound = 0)
TClera = p.LpVariable('TClera', lowBound = 0) 
Stock_ROM = p.LpVariable('Stock_ROM', lowBound = 0) 
Stock_CHPP_CC_135 = p.LpVariable('Stock_CHPP_CC_135', lowBound = 0) 
Stock_CHPP_CC_105 = p.LpVariable('Stock_CHPP_CC_105', lowBound = 0) 
Stock_CHPP_TC = p.LpVariable('Stock_CHPP_TC', lowBound = 0) 
Stock_RS_CC_135 = p.LpVariable('Stock_RS_CC_135', lowBound = 0) 
Stock_RS_CC_105 = p.LpVariable('Stock_RS_CC_105', lowBound = 0) 
Stock_RS_TC = p.LpVariable('Stock_RS_TC', lowBound = 0) 
Stock_Port_CC_135 = p.LpVariable('Stock_Port_CC_135', lowBound = 0) 
Stock_Port_CC_105 = p.LpVariable('Stock_Port_CC_105', lowBound = 0) 
Stock_Port_TC = p.LpVariable('Stock_Port_TC', lowBound = 0) 
ccvessel_135 = p.LpVariable('ccvessel_135', lowBound = 0, cat=p.LpInteger) 
ccvessel_105 = p.LpVariable('ccvessel_105', lowBound = 0, cat=p.LpInteger) 
tcvessel = p.LpVariable('tcvessel', lowBound = 0, cat=p.LpInteger) 


# Objective Function of profit earned per ton / (-) for loss incurred per ton
Lp_prob+= 10 * CCs_135 + 15 * CCs_105 + 15 * TCs   



# DIFFERENT CONSTRAINTS:
#Sales constraint as per EJC
Lp_prob += CCs_135 >= 0 #taking the value >=0 shall give us the best decision without any pressure from EJC
Lp_prob += CCs_105 >= 0
Lp_prob += TCs >= 0 #taking the value >=0 shall give us the best decision without any pressure from EJC

#Minimum Cashflow generation requiremen (Mining cost + CHPP cost + Haulage cost + Rail Cost + Port cost + Other admistrative fixed cost
#Cost is taken as sum of fixed and Variable cost
Lp_prob += 84 * CCs_135 + 100 * CCs_105 + 56 * TCs >= (75*(ROM_mined_perday*28)+ 100000) + (11*(ROM_feed_perday*28)+ 100000) + 3*(CCro_135 + CCro_105 + TCro) + (10.75*(CCra_135 + CCra_105 + CClera_135 + CClera_105) + 8*(TCra+TClera)+100000) + 8.5*(CCs_135+CCs_105+TCs) +(100000)

#Mining constraint
Lp_prob += Stock_ROM + ROM_mined_perday*28 >= ROM_feed_perday*28

#CHPP constraint of yield and working days
Lp_prob += ROM_135_feed_perday * 28 + ROM_105_feed_perday * 28 <= ROM_feed_perday * 28
Lp_prob += ROM_135_feed_perday * 28 + ROM_105_feed_perday * 28 >= ROM_feed_perday * 28

Lp_prob += CCp_135 <= 0.35 * ROM_135_feed_perday * 28
Lp_prob += CCp_135 >= 0.34 * ROM_135_feed_perday * 28
Lp_prob += CCp_105 <= 0.23 * ROM_105_feed_perday * 28
Lp_prob += CCp_105 >= 0.22 * ROM_105_feed_perday * 28
Lp_prob += TCp <= 0.10 * ROM_135_feed_perday * 28 + 0.18 * ROM_105_feed_perday * 28 

#Road haulage constraint
Lp_prob += CCro_135 + CCro_105 + TCro >= 110000
Lp_prob += CCro_135 + CCro_105 + TCro <= 150000

#Rail transport constraint (Rail transport is sum of own railing + lease railing)
Lp_prob += CCra_135 + CCra_105 + TCra >= 116667
Lp_prob += CCra_105 + CCra_105 + TCra <= 160000
Lp_prob += CClera_135 + CClera_105 + TClera <= 30000 #assuming that only one train is taken on lease


#Vessel capacity constraint (Note that <= and >= is done to make the equations EQUAL TO in an indirect way)
Lp_prob += ccvessel_135 <= CCs_135*1/38500
Lp_prob += ccvessel_105 <= CCs_105*1/38500
Lp_prob += tcvessel <= TCs*1/38500
Lp_prob += ccvessel_135 >= CCs_135*1/38500
Lp_prob += ccvessel_105 >= CCs_105*1/38500
Lp_prob += tcvessel >= TCs*1/38500

#Stock flow constraint to avoid stock-out
Lp_prob += Stock_ROM + ROM_mined_perday*28 >= ROM_feed_perday*28

Lp_prob += Stock_CHPP_CC_135 + CCp_135 >= CCro_135
Lp_prob += Stock_CHPP_CC_105 + CCp_105 >= CCro_105
Lp_prob += Stock_CHPP_TC + TCp >= TCro

Lp_prob += Stock_RS_CC_135 + CCro_135 >= CCra_135 + CClera_135
Lp_prob += Stock_RS_CC_105 + CCro_105 >= CCra_105 + CClera_105
Lp_prob += Stock_RS_TC + TCro >= TCra + TClera

Lp_prob += Stock_Port_CC_135 + CCra_135 + CClera_135 >= CCs_135
Lp_prob += Stock_Port_CC_105 + CCra_105 + CClera_105 >= CCs_105
Lp_prob += Stock_Port_TC + TCra + TClera >= TCs

#Minimum stock to be maintained at different stockyards as buffer stock
Lp_prob += Stock_ROM + ROM_mined_perday*28 - ROM_feed_perday*28 >= 100000
Lp_prob += (Stock_CHPP_CC_135 + Stock_CHPP_CC_105 + CCp_135 + CCp_105 - CCro_135 - CCro_105) + (Stock_CHPP_TC + TCp - TCro) >= 40000
Lp_prob += (Stock_RS_CC_135 + Stock_RS_CC_105 + CCro_135 + CCro_105 - CCra_135 - CCra_105 - CClera_135 - CClera_105) + (Stock_RS_TC + TCro - TCra- TClera) >= 40000
Lp_prob += (Stock_Port_CC_135 + Stock_Port_CC_105 + CCra_135 + CCra_105 + CClera_135 + CClera_105 - CCs_135 -CCs_105) + (Stock_Port_TC + TCra + TClera - TCs) >= 40000

#Storage space constraint
Lp_prob += Stock_ROM + ROM_mined_perday *28 - ROM_feed_perday *28 <= 100000 # space at Mine
Lp_prob += Stock_CHPP_CC_135 + Stock_CHPP_CC_105 + Stock_CHPP_TC + CCp_135 +CCp_105 + TCp - CCro_135 - CCro_105 - TCro <= 90000 # space at CHPP  
Lp_prob += Stock_RS_CC_135 + Stock_RS_CC_105 + Stock_RS_TC + CCro_135 + CCro_105 + TCro - CCra_135 - CCra_105 - TCra - CClera_135 - CClera_105 - TClera <= 100000 # space at RS
Lp_prob += Stock_Port_CC_135 + Stock_Port_CC_105 + Stock_Port_TC + CCra_135 + CCra_105 + TCra + CClera_135 + CClera_105 + TClera - CCs_135 - CCs_105 - TCs <=150000 # space at Port

# Opening/Present Stock levels
Lp_prob += Stock_ROM <= 30000
Lp_prob += ROM_mined_perday <= 15000 #from regression
Lp_prob += Stock_CHPP_CC_135 <= 25000
Lp_prob += Stock_CHPP_CC_105 <= 0
Lp_prob += Stock_CHPP_TC <= 600000
Lp_prob += Stock_RS_CC_135 <= 40000
Lp_prob += Stock_RS_CC_105 <= 0
Lp_prob += Stock_RS_TC <= 10000
Lp_prob += Stock_Port_CC_135 <= 60000
Lp_prob += Stock_Port_CC_105 <= 0
Lp_prob += Stock_Port_TC <= 25000

# Display the problem 
print(Lp_prob) 

#printing the status of the optimization problem
status = Lp_prob.solve()   # Solver CBC by default
print('The status of the problem is: ', p.LpStatus[status]) 

# Printing the final solution 
print('Profit value is: ', round(p.value(Lp_prob.objective)))
print('ROM mined is: ', round(p.value(ROM_mined_perday*28)))
print('ROM_feed is: ', round(p.value(ROM_feed_perday*28)))
print('ROM_135_feed is: ', round(p.value(ROM_135_feed_perday*28)))
print('ROM_105_feed is: ', round(p.value(ROM_105_feed_perday*28)))
print('Production in CHPP CC_135: ', round(p.value(CCp_135)), 'CC_105: ', round(p.value(CCp_105)),' TC: ', round(p.value(TCp)))
print('Road despatch CC_135: ', round(p.value(CCro_135)), 'CC_105: ', round(p.value(CCro_105)), 'TC: ', round(p.value(TCro)))
print('Rail despatch CC_135: ', round(p.value(CCra_135)), ' CC_105: ', round(p.value(CCra_105)), 'TC: ', round(p.value(TCra)))
print('Lease Rail despatch CC_135: ', round(p.value(CClera_135)), 'CC_105: ', round(p.value(CClera_105)), 'TC: ', round(p.value(TClera)))
print('Number of vessels CC_135: ', p.value(ccvessel_135), 'CC_105: ', p.value(ccvessel_105), 'TC: ', p.value(tcvessel))
print('Shipping/Sale CC_135: ', round(p.value(CCs_135)), 'CC_105: ', round(p.value(CCs_105)), 'TC: ', round(p.value(TCs)))

print('Stock at Mine  ROM: ', round(p.value(Stock_ROM + ROM_mined_perday*28 - ROM_feed_perday*28)))
print('Stock at CHPP  CC: ', round(p.value(Stock_CHPP_CC_135 + CCp_135 - CCro_135 + Stock_CHPP_CC_105 + CCp_105 - CCro_105 )), 'TC: ', round(p.value(Stock_CHPP_TC + TCp-TCro)))
print('Stock at RS  CC: ', round(p.value(Stock_RS_CC_135 + CCro_135 - CCra_135 - CClera_135 + Stock_RS_CC_105 + CCro_105 - CCra_105 - CClera_105)), 'TC: ', round(p.value(Stock_RS_TC + TCro-TCra-TClera)))
print('Stock at Port  CC: ', round(p.value(Stock_Port_CC_135 + CCra_135 + CClera_135 - CCs_135 + Stock_Port_CC_105 + CCra_105 + CClera_105 - CCs_105)), 'TC: ', round(p.value(Stock_Port_TC + TCra +TClera - TCs)))

Problem:
MAXIMIZE
15*CCs_105 + 10*CCs_135 + 15*TCs + 0
SUBJECT TO
_C1: CCs_135 >= 0

_C2: CCs_105 >= 0

_C3: TCs >= 0

_C4: - 10.75 CClera_105 - 10.75 CClera_135 - 10.75 CCra_105 - 10.75 CCra_135
 - 3 CCro_105 - 3 CCro_135 + 91.5 CCs_105 + 75.5 CCs_135 - 308 ROM_feed_perday
 - 2100 ROM_mined_perday - 8 TClera - 8 TCra - 3 TCro + 47.5 TCs >= 400000

_C5: - 28 ROM_feed_perday + 28 ROM_mined_perday + Stock_ROM >= 0

_C6: 28 ROM_105_feed_perday + 28 ROM_135_feed_perday - 28 ROM_feed_perday <= 0

_C7: 28 ROM_105_feed_perday + 28 ROM_135_feed_perday - 28 ROM_feed_perday >= 0

_C8: CCp_135 - 9.8 ROM_135_feed_perday <= 0

_C9: CCp_135 - 9.52 ROM_135_feed_perday >= 0

_C10: CCp_105 - 6.44 ROM_105_feed_perday <= 0

_C11: CCp_105 - 6.16 ROM_105_feed_perday >= 0

_C12: - 5.04 ROM_105_feed_perday - 2.8 ROM_135_feed_perday + TCp <= 0

_C13: CCro_105 + CCro_135 + TCro >= 110000

_C14: CCro_105 + CCro_135 + TCro <= 150000

_C15: CCra_105 + CCra_135 + TCra >= 116667

_C16: 2 CCra_105 + TCra <= 160000

